## 버트(BERT)

2018년 11월, 구글이 공개한 인공지능 언어 모델 BERT(Bidirectional Encoder Representations from Transformers)는 기존의 단방향 자연어 처리 모델들의 단점을 보완한 양방향 자연어 처리 모델입니다. 검색 문장의 단어를 입력된 순서대로 하나씩 처리하는 것이 아니라, 트랜스포머를 이용하여 구현되었으며 방대한 양의 텍스트 데이터로 사전 훈련된 언어 모델입니다.

![](../Static/587.jpg)

버트의 기본 구조는 그림과 같이 트랜스포머(transformer)라는 인코더를 쌓아 올린 구조로, 주로 `문장 예측(NSP)`을 할 때 사용합니다. 튜닝을 통해 최고의 성능을 낸 기존 사례들을 참고해서 사전 학습된 버트 위에 분류를 위한 신경망을 한층 추가하는 방식을 사용합니다. 즉, 버트는 트랜스포머와 사전 학습을 사용하여 성능을 향상시킨 모델입니다.

버트의 학습 절차는 다음과 같습니다.

1. 그림과 같이 문장을 버트의 입력 형식에 맞게 변환합니다. 이때 문장의 시작은 [CLS], 문장의 끝은 [SEP]로 표시합니다.

2. 한 문장의 단어들에 대해 `토큰화(tokenization)`을 진행합니다. 예를 들어 '고양이'라는 단어의 경우 '고##','#양#','##이'로 토큰화합니다.

3. 마지막으로 각 토큰들에 대해 고유의 아이디를 부여합니다. 토큰이 존재하지 않는 자리는 0으로 채웁니다.

버트 모델은 전이 학습을 기반으로 한다고 했는데, 이떄 전이는 인코더-디코더로 된 모델입니다. 기존 인코더-디코더 모델들과 다르게 CNN,RNN을 이용하지 않고 `어텐션 개념`을 도입했습니다. 즉, 버트에서 전이 학습은 인코더-디코더중 `인코더만 사용`하는 모델입니다.

버트는 두 가지 버전이 있는데, BERT-base(L=12, H=768, A=12)와 BERT-large(L=24, H=1024, A=16)입니다. 이떄 L은 전이 블록 숫자이고, H는 은닉층 크기, A는 전이 블록에서 사용되는 어텐션 블록 숫자입니다. 즉, L,H,A가 크다는 것은 블록을 많이 쌓았고, 표현하는 은닉층이 크며 어텐션 개수를 많이 사용했다는 의미입니다. BERT-base는 학습 파라미터가 1.1억개가 있고, BERT-large는 학습 파라미터가 3.4억개 있습니다.

예제를 진행하기에 앞서 파이토치에서 버트를 사용하기 위한 라이브러리를 설치합니다.

> pip install transformers

> pip install pytorch-transformers

예제에서 사용하는 데이터셋은 네이버 영화 리뷰입니다. 데이터셋은 다음 URL에서도 내려받을 수 있습니다.

https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix # 모델 평가를 위해 사용
import seaborn as sns

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
train_df = pd.read_csv('./rating/ratings.train.txt', sep='\t')
test_df = pd.read_csv('./rating/ratings.test.txt', sep='\t')
train_df